In [14]:
import os

# Function to count objects by ID in YOLO annotations
def count_objects_by_id(yolo_annotations_dir):
    object_count = {}

    # Read each YOLO annotation file
    for filename in os.listdir(yolo_annotations_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(yolo_annotations_dir, filename), 'r') as file:
                lines = file.readlines()

                for line in lines:
                    class_id = int(line.strip().split()[0])

                    if class_id in object_count:
                        object_count[class_id] += 1
                    else:
                        object_count[class_id] = 1

    return object_count

# Example usage:
yolo_annotations_dir = r"C:\Users\M\OneDrive - UGent\TWML\datasets\sthsch_old\Curated_Old_STHSCH\valid_\labels"

object_counts = count_objects_by_id(yolo_annotations_dir)
print("Object Counts by ID:")
for class_id, count in object_counts.items():
    print(f"Class ID {class_id}: {count} objects")


Object Counts by ID:
Class ID 0: 1766 objects
Class ID 1: 904 objects
Class ID 2: 760 objects
Class ID 3: 143 objects


In [8]:
import os
import json
import numpy as np
from PIL import Image

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height

# Function to convert YOLO annotations to COCO format
def convert_yolo_to_coco(yolo_annotations_dir, images_dir, output_json_path, categories):
    images = []
    annotations = []

    image_id = 0
    annotation_id = 0

    # Read each YOLO annotation file
    for filename in os.listdir(yolo_annotations_dir):
        if filename.endswith(".txt"):
            image_id += 1
            image_filename = filename[:-4] + '.jpeg'
            image_path = os.path.join(images_dir, image_filename)
            
            # Get image dimensions
            width, height = get_image_dimensions(image_path)
            
            image_info = {'id': image_id, 'file_name': image_filename, 'width': width, 'height': height}

            images.append(image_info)

            with open(os.path.join(yolo_annotations_dir, filename), 'r') as file:
                lines = file.readlines()

                for line in lines:
                    line = line.strip().split()
                    class_id = int(line[0])
                    x_center = float(line[1])
                    y_center = float(line[2])
                    bbox_width = float(line[3])
                    bbox_height = float(line[4])

                    x_min = (x_center - bbox_width / 2) * width
                    y_min = (y_center - bbox_height / 2) * height
                    x_max = (x_center + bbox_width / 2) * width
                    y_max = (y_center + bbox_height / 2) * height

                    annotation_info = {
                        'id': annotation_id,
                        'image_id': image_id,
                        'category_id': class_id,
                        'bbox': [x_min, y_min, x_max - x_min, y_max - y_min],
                        'area': (x_max - x_min) * (y_max - y_min),
                        'iscrowd': 0
                    }

                    annotations.append(annotation_info)
                    annotation_id += 1

    # Create COCO format dictionary
    coco_format = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }

    # Save to JSON file
    with open(output_json_path, 'w') as outfile:
        json.dump(coco_format, outfile, indent=4)

    print(f"Conversion completed. COCO format annotations saved to: {output_json_path}")



In [15]:
# Example usage:
yolo_annotations_dir = r"C:\Users\M\OneDrive - UGent\TWML\datasets\sthsch_old\Curated_Old_STHSCH\valid_\labels"
images_dir = r"C:\Users\M\OneDrive - UGent\TWML\datasets\sthsch_old\Curated_Old_STHSCH\valid_\images"
output_json_path = r"C:\Users\M\OneDrive - UGent\TWML\datasets\sthsch_old\Curated_Old_STHSCH\valid_\valid.json"
categories = [{'id': 0, 'name': 'ascaris'}, 
              {'id': 1, 'name': 'trichuris'}, 
              {'id': 2, 'name': 'hookworm'},
              {'id': 3, 'name': 'schistosoma'}]  # Replace with your own categories

convert_yolo_to_coco(yolo_annotations_dir, images_dir, output_json_path, categories)


Conversion completed. COCO format annotations saved to: C:\Users\M\OneDrive - UGent\TWML\datasets\sthsch_old\Curated_Old_STHSCH\valid_\valid.json


In [16]:
import json

# Function to count objects by category ID in COCO format annotations
def count_objects_by_category(coco_annotations_path):
    # Load COCO annotations JSON file
    with open(coco_annotations_path, 'r') as f:
        coco_data = json.load(f)

    category_counts = {}

    # Initialize category counts
    for category in coco_data['categories']:
        category_counts[category['id']] = 0

    # Count objects by category ID
    for annotation in coco_data['annotations']:
        category_id = annotation['category_id']
        category_counts[category_id] += 1

    return category_counts

# Example usage:
coco_annotations_path = r"C:\Users\M\OneDrive - UGent\TWML\datasets\sthsch_old\Curated_Old_STHSCH\valid_\valid.json"

category_counts = count_objects_by_category(coco_annotations_path)

# Print counts
for category_id, count in category_counts.items():
    print(f"Category ID {category_id}: {count} objects")


Category ID 0: 1766 objects
Category ID 1: 904 objects
Category ID 2: 760 objects
Category ID 3: 143 objects
